In [ ]:
'''
Script: Accessibility Model Ver.3
This arcpy tool allows the user to calculate drive times, miles, number of turn right and left, traffic volumes
decays from starting points in major roads in US with 0.25 miles distance to a target destination. It uses one 
csv file, which has market name, latitude, and longitude, as an input. With the lat, long values, it locates
site and major road points. Also this tool allows users to customize departure time and will create each routing 
lines to destinations. The summarized final csv file will be created in the workfolder.
Author: Jeong Hoon Kim (JeongHoon.Kim@cbre.com)
Date: May 25, 2022
Last Updated: April 20, 2023
'''
# 0. Environment
# Import modules
import arcpy, os, sys
from arcpy import env
import itertools
import pandas as pd
import numpy as np
import functools as ft
import datetime
# Environment options
arcpy.CheckOutExtension("network")
arcpy.env.overwriteOutput = True
# Set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
title_name = arcpy.GetParameterAsText(1) # workspace
trgt_csv = arcpy.GetParameterAsText(2) # target site csv, Table or Table view
lat_field = arcpy.GetParameterAsText(3) # latitude field, target site csv
long_field = arcpy.GetParameterAsText(4) # long field, target site csv
buff_dis = arcpy.GetParameterAsText(5) # "0.25 Miles" Default
depart_time = arcpy.GetParameterAsText(6) # date & time
# Define project, map, and workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs
aprx = arcpy.mp.ArcGISProject("CURRENT") # Current Project
aprxMap = aprx.listMaps("Map")[0] # Dafault Map
#workfolder = os.getcwd()

# Check input file
isItCSV_trgt = trgt_csv[-4:]
if isItCSV_trgt == '.csv':
    arcpy.AddMessage("Entered a csv for market sites!")
    try: 
        # 1. Create analysis objects (sites, buffer, roads)
        # Create a Feature Dataset
        arcpy.management.CreateFeatureDataset(out_dataset_path = workspace, 
                                    out_name = title_name,
                                    spatial_reference = arcpy.SpatialReference("WGS 1984"))
        accessbility_fd = os.path.join(workspace, title_name)
        # Create unprojected feature layer for office sites
        market_p = os.path.join(accessbility_fd, title_name+"_Markets_Original")
        arcpy.management.XYTableToPoint(
            in_table = trgt_csv, 
            out_feature_class = market_p, 
            x_field = long_field, 
            y_field = lat_field, 
            coordinate_system = arcpy.SpatialReference("WGS 1984"))
        # Create Buffer
        buffer = os.path.join(accessbility_fd, title_name+"_Buffers")
        arcpy.Buffer_analysis(market_p, buffer, buff_dis) 
        # Create intersections between the buffer and major roads feature layer
        major_roads = r"C:\ArcGIS\Business Analyst\US_2022\Data\Streets Data\NorthAmerica.gdb\Streets"
        start_multi = os.path.join(accessbility_fd, title_name+"_Start_Multi")
        arcpy.Intersect_analysis([buffer, major_roads], start_multi, "", "", "point")
        # Converted to signlepar tpoint by using the Multipart to Singlepart tool
        start_p = os.path.join(accessbility_fd, title_name+"_AllStreets")
        arcpy.MultipartToSinglepart_management(start_multi, start_p)
        arcpy.management.Delete(start_multi)
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 1 | created sites, roads, and buffers!")
    
    try: 
        # Set NetworkDataset object variables to make closest facility analysis layer
        input_facilities = market_p
        input_incidents = start_p
        output_routes = os.path.join(accessbility_fd, title_name+"_Routes_Original")
        output_directions = os.path.join(accessbility_fd, title_name+"_Directions")
        # Create a network dataset layer and get the desired travel mode for analysis
        #arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
        nd_travel_modes = arcpy.nax.GetTravelModes("https://www.arcgis.com/")
        travel_mode = nd_travel_modes["Driving Time"]
        # Instantiate a ClosestFacility solver object
        closest_facility = arcpy.nax.ClosestFacility("https://www.arcgis.com/")
        # Set properties
        closest_facility.travelMode = travel_mode
        closest_facility.timeUnits = arcpy.nax.TimeUnits.Minutes
        closest_facility.defaultTargetFacilityCount = 1
        closest_facility.routeShapeType = arcpy.nax.RouteShapeType.TrueShapeWithMeasures
        closest_facility.returnDirections = True
        closest_facility.timeOfDay = datetime.datetime.strptime(depart_time, '%m/%d/%Y %I:%M:%S %p')
        closest_facility.timeOfDayUsage = arcpy.nax.TimeOfDayUsage.DepartureTime
        closest_facility.timeZone = arcpy.nax.TimeZoneUsage.UTC
        closest_facility.travelDirection = arcpy.nax.TravelDirection.ToFacility
        closest_facility.distanceUnits = arcpy.nax.DistanceUnits.Miles
        closest_facility.directionsDistanceUnits = arcpy.nax.DistanceUnits.Miles
        # Load inputs
        closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Facilities, input_facilities)
        closest_facility.load(arcpy.nax.ClosestFacilityInputDataType.Incidents, input_incidents)
        # Solve the analysis
        result = closest_facility.solve()
        # Export the results to a feature class
        if result.solveSucceeded:
            result.export(arcpy.nax.ClosestFacilityOutputDataType.Routes, output_routes)
            result.export(arcpy.nax.ClosestFacilityOutputDataType.Directions, output_directions)
        else:
            arcpy.AddMessage("Solve failed")
            arcpy.AddMessage(result.solverMessages(arcpy.nax.MessageSeverity.All))
        # Split Name fields with Facility and Incident
        arcpy.AddField_management(output_routes, "Name_MarketID", "TEXT")
        arcpy.AddField_management(output_routes, "Name_RoadID", "TEXT")
        with arcpy.da.UpdateCursor(output_routes, ["Name", "Name_RoadID", "Name_MarketID"]) as cursor:
            for row in cursor:
                row[1] = row[0].split(" - ")[0]
                row[2] = row[0].split(" - ")[1]
                cursor.updateRow(row)
        # Select routes having same 'Name_RoadID' and 'Name_MarketID' and copy selected features
        routes_selected = arcpy.management.SelectLayerByAttribute(output_routes, 'NEW_SELECTION', 'Name_RoadID = Name_MarketID')
        routes_filtered = os.path.join(accessbility_fd, title_name+"_Routes_Filtered")
        arcpy.management.CopyFeatures(routes_selected, routes_filtered)
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 2 | created routes and directions!")

    try: 
        # Capture the closest traffic counts from road points with spatial join
        sl_traffics = r"Q:\Everyone\Kim_JH\Projects\Accessibility\Accessibility.gdb\StreetLight_Traffic"
        routes_traffics = os.path.join(accessbility_fd, title_name+"_Routes_Traffics")
        arcpy.analysis.SpatialJoin(routes_filtered, sl_traffics, routes_traffics, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "CLOSEST_GEODESIC", "", "")
        arcpy.management.Delete(routes_filtered)
        # Create 'Traffic Decays' and 'Traffic Scores'
        arcpy.AddField_management(routes_traffics, "Traffic_Decays", "DOUBLE")
        arcpy.AddField_management(routes_traffics, "Traffic_Scores", "DOUBLE")
        arcpy.management.CalculateField(routes_traffics, "Traffic_Decays", "!TRAFFIC1! / !Total_Minutes!", "PYTHON3")
        arcpy.management.CalculateField(routes_traffics, "Traffic_Scores", "(1 / !Total_Miles!) + (!TRAFFIC1! * 0.2)", "PYTHON3")
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 3 | captured traffics!")

    try:
        # Convert feature layer to table
        arcpy.TableToTable_conversion(routes_traffics, workspace, title_name+"_RoutesTraf_Table")
        arcpy.TableToTable_conversion(output_directions, workspace, title_name+"_Directions_Table")
        # Convert gdb table to dataframe in pandas
        routesTraf_table = os.path.join(workspace, title_name+"_RoutesTraf_Table")
        routesTraf_field = [f.name for f in arcpy.ListFields(routesTraf_table)]
        routesTraf_array = arcpy.da.TableToNumPyArray(routesTraf_table, routesTraf_field)
        routesTraf_df = pd.DataFrame(routesTraf_array, columns = routesTraf_field)
        directions_table = os.path.join(workspace, title_name+"_Directions_Table")
        directions_field = [f.name for f in arcpy.ListFields(directions_table)]
        directions_array = arcpy.da.TableToNumPyArray(directions_table, directions_field)
        directions_df = pd.DataFrame(directions_array, columns = directions_field)
        # Create 'Route ID' in directions_df
        # Convert 'Type' column to string
        directions_df['Type'] = directions_df['Type'].astype(str)
        # Create definition to create 'Route ID'
        def route_id(row):
            if '18' in row['Type']:
                val = 1
            else:
                val = 0
            return val
        directions_df['Route_ID'] = directions_df.apply(route_id, axis=1)
        directions_df['Route_ID'] = directions_df['Route_ID'].cumsum()
        # Filter the direction with 'Route ID'
        route_filtered_id = routesTraf_df['IncidentOID'].values.tolist()
        directions_filtered_df = directions_df[directions_df["Route_ID"].isin(route_filtered_id)] # 1953 / 2156
        # Create 'Turn Right' and 'Turn Left' in directions_df
        # Create definition to create the columns
        def turn_left1(row):
            if 'turn left' in row['Text']:
                val = 1
            elif 'Turn left' in row['Text']:
                val = 1
            elif 'sharp left' in row['Text']:
                val = 1
            elif 'Bear left' in row['Text']:
                val = 1
            elif 'bear left' in row['Text']:
                val = 1
            else:
                val = 0
            return val
        def turn_left2(row):
            if 'U-turn' in row['Text']:
                val = 2
            elif 'Turn left, then turn left' in row['Text']:
                val = 2
            else:
                val = row['Turn_Left']
            return val
        def turn_right1(row):
            if 'turn right' in row['Text']:
                val = 1
            elif 'Turn right' in row['Text']:
                val = 1
            elif 'sharp right' in row['Text']:
                val = 1
            elif 'Bear right' in row['Text']:
                val = 1
            elif 'bear right' in row['Text']:
                val = 1
            else:
                val = 0
            return val
        def turn_right2(row):
            if 'Turn right, then turn right' in row['Text']:
                val = 2
            else:
                val = row['Turn_Right']
            return val
        directions_df['Turn_Left'] = directions_df.apply(turn_left1, axis=1)
        directions_df['Turn_Left'] = directions_df.apply(turn_left2, axis=1)
        directions_df['Turn_Right'] = directions_df.apply(turn_right1, axis=1)
        directions_df['Turn_Right'] = directions_df.apply(turn_right2, axis=1)
        # Create a df with number of turn right and left by route id
        summary_fields = ['Turn_Left', 'Turn_Right']
        routesTurn_df = directions_df.groupby(['Route_ID'], as_index=False)[summary_fields].sum()
        # Convert df to gdb table
        routesTurn_records = routesTurn_df.to_records(index=False)
        routesTurn_array = np.array(routesTurn_records, dtype = routesTurn_records.dtype.descr)
        routesTurn_table = os.path.join(workspace, title_name+"_RoutesTurn_Table")
        arcpy.da.NumPyArrayToTable(routesTurn_array, routesTurn_table)
        # Delte unnecessary objects
        arcpy.management.Delete(routesTraf_table)
        arcpy.management.Delete(directions_table)
        # Join turn table to feature layer
        # Add index to 'Routes_Traffics'
        route_joined_fc = arcpy.management.AddJoin(routes_traffics, "IncidentOID", routesTurn_table, "Route_ID", "KEEP_COMMON")
        routes_traffics_turns = os.path.join(accessbility_fd, title_name+"_Routes_TrafficsTurns")
        arcpy.management.CopyFeatures(route_joined_fc, routes_traffics_turns)
        # Delte unnecessary objects
        arcpy.management.Delete(routes_traffics)
        arcpy.management.Delete(routesTurn_table)
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 4 | claculated turns!")

    try: 
        # Generate summary statistics
        access_summary_table = os.path.join(workspace, title_name+"_AccessSummary_Table")
        field_mile = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Total_Miles*")][0]
        field_minute = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Total_Minutes*")][0]
        field_right = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Turn_Right*")][0]
        field_left = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Turn_Left*")][0]
        field_trafficD = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Traffic_Decays*")][0]
        field_trafficS = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*Traffic_Scores*")][0]
        field_case = [f.name for f in arcpy.ListFields(routes_traffics_turns, "*FacilityOID*")][0]
        statistic_fields = [[field_mile, "MEAN"], [field_mile, "MAX"], [field_mile, "MIN"], [field_mile, "RANGE"], [field_mile, "STD"],
                            [field_minute, "MEAN"], [field_minute, "MAX"], [field_minute, "MIN"], [field_minute, "RANGE"], [field_minute, "STD"],
                            [field_right, "MEAN"], [field_right, "MAX"], [field_right, "MIN"], [field_right, "RANGE"], [field_right, "STD"],
                            [field_left, "MEAN"], [field_left, "MAX"], [field_left, "MIN"], [field_left, "RANGE"], [field_left, "STD"],
                            [field_trafficD, "MEAN"], [field_trafficD, "MAX"], [field_trafficD, "MIN"], [field_trafficD, "RANGE"], [field_trafficD, "STD"],
                            [field_trafficS, "MEAN"], [field_trafficS, "MAX"], [field_trafficS, "MIN"], [field_trafficS, "RANGE"], [field_trafficS, "STD"]]
        case_field = field_case
        arcpy.analysis.Statistics(routes_traffics_turns, access_summary_table, statistic_fields, case_field)
        # Join summary table to market fc
        market_summary = os.path.join(accessbility_fd, title_name+"_Markets_Summary")
        market_joined_fc = arcpy.management.AddJoin(market_p, "OBJECTID", access_summary_table, field_case, "KEEP_ALL")
        arcpy.management.CopyFeatures(market_joined_fc, market_summary)
        # Delte unnecessary objects
        arcpy.management.Delete(access_summary_table)
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 5 | generated summary statistics!")
    
    try: 
        # Add selected features to the current map
        import_fcs = [buffer, routes_traffics_turns, start_p, market_summary] 
        for fc in import_fcs:
            aprxMap.addDataFromPath(fc)
    except arcpy.ExecuteError:
        arcpy.GetMessages(2)
    else:
        # Add arcpy message
        arcpy.AddMessage("STEP 6 | Added to the current map!")
        
elif len(trgt_csv) == 0:
    arcpy.AddMessage("Did not enter a csv file for target locations!")
else:
    arcpy.AddMessage("Entered a wrong format for target locations!")
    pass